## Repressilator
Example to test handling rational functions (such as hill functions) in Oscar.

In [ ]:
using CRNAnalysis
using CRNAnalysis: polynomial_ideal
using Catalyst
using Oscar
import Symbolics

In [116]:
function repressilator(n::Int; name=:Repressilator)
    t = default_t()
    @reaction_network $name begin
        @species X(t) Y(t) Z(t)
        hillr(Z, β, 1, $n), 0 --> X
        hillr(X, β, 1, $n), 0 --> Y
        hillr(Y, β, 1, $n), 0 --> Z
        1, X --> 0
        1, Y --> 0
        1, Z --> 0
    end
end
rn = repressilator(2)
nlsys = convert(NonlinearSystem, rn)

Model Repressilator:
Equations (3):
  3 standard: see equations(Repressilator)
Unknowns (3): see unknowns(Repressilator)
  X(t)
  Y(t)
  Z(t)
Parameters (1): see parameters(Repressilator)
  β

Steps:
- check if mass-action/polynomial
  - if all reactions are, continue as normal
- if not, check for rational functions
  - convert to polynomials
    - factor out denominator of rational functions (LHS = 0)
- otherwise, error

In [155]:
is_rational_function(f) = !isequal(denominator(f), 1)

implicit_form(eqn::Equation) = isequal(eqn.rhs, 0) ? eqn.lhs : (eqn.rhs - eqn.lhs)

# rational equation --> polynomial equation
function expand_rational_equation(eqn::Equation)
    f = implicit_form(eqn)
    # collect denominators
    denoms = [denominator(term)
        for term in Symbolics.terms(f)
        if is_rational_function(term)
    ]
    # expand and simplify
    new_f = f * prod(denoms)
    Symbolics.simplify(new_f, expand=true)
end

function symbolic_function(eqn_rhs, vars, params)
    Symbolics.build_function(eqn_rhs, vars, params; expression=Val{false})[1]
end

symbolic_function (generic function with 1 method)

In [120]:
equations(nlsys)

3-element Vector{Equation}:
 0 ~ -X(t) + β / (1 + Z(t)^2)
 0 ~ -Y(t) + β / (1 + X(t)^2)
 0 ~ -Z(t) + β / (1 + Y(t)^2)

In [121]:
polyn_rates = map(equations(nlsys)) do eqn
    expand_rational_equation(eqn)
end

3-element Vector{Num}:
 -X(t) + β - X(t)*(Z(t)^2)
 -Y(t) + β - Y(t)*(X(t)^2)
 -Z(t) + β - (Y(t)^2)*Z(t)

In [122]:
vars = Catalyst.unknowns(nlsys)
params = parameters(nlsys)
I = polynomial_ideal(polyn_rates, vars, params)

MethodError: MethodError: no method matching (::AbstractAlgebra.Generic.MPolyRing{AbstractAlgebra.Generic.FracFieldElem{QQMPolyRingElem}})(::Num)
The object of type `AbstractAlgebra.Generic.MPolyRing{AbstractAlgebra.Generic.FracFieldElem{QQMPolyRingElem}}` exists, but no method is defined for this combination of argument types when trying to treat it as a callable object.

Closest candidates are:
  (::AbstractAlgebra.Generic.MPolyRing{T})(!Matched::Vector{T}, !Matched::Vector{Vector{Int64}}) where T<:RingElement
   @ AbstractAlgebra ~/.julia/packages/AbstractAlgebra/L8iQ0/src/generic/MPoly.jl:4069
  (::AbstractAlgebra.Generic.MPolyRing{T})(!Matched::Vector{T}, !Matched::Matrix{UInt64}) where T<:RingElement
   @ AbstractAlgebra ~/.julia/packages/AbstractAlgebra/L8iQ0/src/generic/MPoly.jl:4058
  (::AbstractAlgebra.Generic.MPolyRing{T})(!Matched::Singular.spoly{Singular.n_RingElem{T}}) where T<:RingElem
   @ Singular ~/.julia/packages/Singular/QXKD9/src/poly/poly.jl:1221
  ...


In [123]:
# Convert Catalyst symbolic variables into Julia Symbol types
varnames = tosymbol.(vars, escape=false)
paramnames = tosymbol.(params)

1-element Vector{Symbol}:
 :β

In [124]:

# Create polynomial ring in Oscar
CC, oscar_coeffs = polynomial_ring(QQ, paramnames)
ff = fraction_field(CC)
RR, oscar_vars = polynomial_ring(ff, varnames)

(Multivariate polynomial ring in 3 variables over ff, AbstractAlgebra.Generic.MPoly{AbstractAlgebra.Generic.FracFieldElem{QQMPolyRingElem}}[X, Y, Z])

In [125]:
# Map Catalyst variables to Oscar variables
cat_var_params = [vars; params]
oscar_var_params = [oscar_vars; oscar_coeffs]
cat_to_oscar = Dict(cat => oscar for (cat, oscar) in zip(cat_var_params, oscar_var_params))
oscar_to_cat = Dict((oscar => cat) for (cat, oscar) in cat_to_oscar)

Dict{MPolyRingElem, SymbolicUtils.BasicSymbolic{Real}} with 4 entries:
  Y => Y(t)
  Z => Z(t)
  X => X(t)
  β => β

In [138]:
polyn_rates

3-element Vector{Num}:
 -X(t) + β - X(t)*(Z(t)^2)
 -Y(t) + β - Y(t)*(X(t)^2)
 -Z(t) + β - (Y(t)^2)*Z(t)

In [158]:
f = symbolic_function(polyn_rates, vars, params)
polys = f(oscar_vars, oscar_coeffs)

3-element Vector{AbstractAlgebra.Generic.MPoly{AbstractAlgebra.Generic.FracFieldElem{QQMPolyRingElem}}}:
 -X*Z^2 - X + β
 -X^2*Y - Y + β
 -Y^2*Z - Z + β

In [159]:

# build Oscar polynomial by substituting oscar vars in Catalyst equations RHS (Right Hand Sides)

ideal(RR, polys)

Ideal generated by
  -X*Z^2 - X + β
  -X^2*Y - Y + β
  -Y^2*Z - Z + β